In [1]:
from MQGO import data, main, ML_engine, optimizer, QC_engine 

In [2]:
from deepmd.calculator import DP

2022-04-19 18:21:38.775744: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-19 18:21:38.775788: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
setting = {'basis':'6-31g', 'spin':0, 'xc':None, 'restricted':True, 'verbose':4}
geo = main.MLgeomopt(qc_engine='pyscf', ml_engine='deepmd', work_path='./', xyz_path='./water.xyz',**setting)
geo.opt_algorithm = 'lbfgs'
geo.kernel()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-12
SCF conv_tol_grad = None
SCF max_cycles = 100
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/jingheng/ML_QC_GEOM_OPT/test/water/dimer/tmpby6g66nu
max_memory 4000 MB (current use 356 MB)
Set gradient conv threshold to 1e-06


OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-15
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #216: KMP_AFFINITY: x2APIC ids not unique.
OMP: Info #216: KMP_AFFINITY: decoding legacy APIC ids.
OMP: Info #216: KMP_AFFINITY: legacy APIC ids not unique.
OMP: Info #220: KMP_AFFINITY: parsing /proc/cpuinfo.
OMP: Info #157: KMP_AFFINITY: 16 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #287: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #192: KMP_AFFINITY: 1 socket x 8 cores/socket x 2 threads/core (8 total cores)
OMP: Info #218: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 1 maps to socket 0 core 0 thread 1 
OMP: Info #172: KMP_AFFINITY: OS proc 2 maps to socket 0 core 1 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 3 maps to socket 0 core 1 thread 1 
OMP: Info #172: KMP_AFFINITY: OS p

init E= -151.415122555485
  HOMO = -0.458914611898629  LUMO = 0.101611688525219
cycle= 1 E= -151.868630726732  delta_E= -0.454  |g|= 0.597  |ddm|= 1.64
  HOMO = -0.400323409471753  LUMO = 0.194953131082916
cycle= 2 E= -151.927771207761  delta_E= -0.0591  |g|= 0.357  |ddm|= 0.608
  HOMO = -0.499862289875181  LUMO = 0.180933357043698
cycle= 3 E= -151.95036021667  delta_E= -0.0226  |g|= 0.0355  |ddm|= 0.229
  HOMO = -0.497612927938669  LUMO = 0.187865526656983
cycle= 4 E= -151.950856808118  delta_E= -0.000497  |g|= 0.00594  |ddm|= 0.0375
  HOMO = -0.498863887981172  LUMO = 0.187642464715892
cycle= 5 E= -151.950875581764  delta_E= -1.88e-05  |g|= 0.000688  |ddm|= 0.00925
  HOMO = -0.498920556933567  LUMO = 0.187620019572834
cycle= 6 E= -151.950875817958  delta_E= -2.36e-07  |g|= 0.000198  |ddm|= 0.00111
  HOMO = -0.498975213195489  LUMO = 0.18761647754511
cycle= 7 E= -151.950875827637  delta_E= -9.68e-09  |g|= 5.74e-05  |ddm|= 0.000157
  HOMO = -0.498963904179646  LUMO = 0.187615348330249


2022-04-19 18:21:40.823680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-19 18:21:40.823720: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
/home/jingheng/venv/lib/python3.9/site-packages/deepmd/utils/data_system.py:158: UserWarning: system ./raw/ required batch size is larger than the size of the dataset raw/set.000 (6 > 1)
  warnings.warn("system %s required batch size is larger than the size of the dataset %s (%d > %d)" % \
2022-04-19 18:21:44.122956: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Ne

optimizer.py 77: [[ 3.63802  1.5711   1.76   ]
 [26.36198  1.5711   1.76   ]
 [ 3.95828  2.55412  1.74392]
 [ 2.63613  1.66728  1.76493]
 [26.09174  0.99418  2.56104]
 [26.0955   0.9908   0.96072]]


...

optimizer.py 77: [[ 3.4108103   1.93083302  1.75628333]
 [26.18307341  1.18536016  1.76058668]
 [ 4.08694778  2.5615632   1.74300367]
 [ 2.73467193  1.30010376  1.76956301]
 [26.15835841  1.13869862  2.68382592]
 [26.20778818  1.23202122  0.83734741]]
       Step     Time          Energy          fmax
LBFGS:    0 18:42:47    -4132.938368         0.135182
LBFGS:    1 18:42:47    -4132.939898         0.260720
LBFGS:    2 18:42:47    -4132.937813         0.002030
LBFGS:    3 18:42:47    -4132.937813         0.000044
main.py 94: QC_coords1 [[ 6.44549703  3.64874545  3.3188945 ]
 [49.47883783  2.24000602  3.32702665]
 [ 7.7203816   4.83801253  3.29385516]
 [ 5.17061167  2.45948056  3.34393385]
 [49.43223674  2.15202389  5.06783097]
 [49.52543853  2.32798736  1.58622226]]


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-12
SCF 

2022-04-19 18:42:47.653662: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-19 18:42:47.653705: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
2022-04-19 18:42:50.930265: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-19 18:42:50.933374: W tensorflow/stream_executor/platform/default/dso_loader

optimizer.py 77: [[ 3.41081014  1.93083294  1.75628333]
 [26.1830734   1.18536014  1.76058668]
 [ 4.08545     2.56016598  1.74303309]
 [ 2.73616986  1.30150106  1.76953359]
 [26.15841317  1.138802    2.68178066]
 [26.20773343  1.23191786  0.83939267]]
       Step     Time          Energy          fmax
LBFGS:    0 18:46:07    -4133.012082         0.147517
LBFGS:    1 18:46:07    -4133.013930         0.291347
LBFGS:    2 18:46:08    -4133.011436         0.002349
LBFGS:    3 18:46:08    -4133.011436         0.000736
LBFGS:    4 18:46:08    -4133.011436         0.002676
LBFGS:    5 18:46:08    -4133.011436         0.000003
main.py 94: QC_coords1 [[ 6.44549715  3.64874576  3.31889449]
 [49.4788378   2.24000597  3.32702664]
 [ 7.72341481  4.84084195  3.29379559]
 [ 5.16757833  2.45665083  3.34399343]
 [49.43212588  2.15181459  5.07197279]
 [49.52554942  2.32819672  1.58208044]]


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift fa

2022-04-19 18:46:08.612200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-19 18:46:08.612242: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Instructions for updating:
non-resource variables are not supported in the long term
DEEPMD INFO    Calculate neighbor statistics... (add --skip-neighbor-stat to skip this step)
2022-04-19 18:46:11.906417: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-19 18:46:11.909473: W tensorflow/stream_executor/platform/default/dso_loader

optimizer.py 77: [[ 3.4108102   1.93083311  1.75628333]
 [26.18307339  1.18536011  1.76058668]
 [ 4.08705511  2.56166324  1.74300156]
 [ 2.73456469  1.30000363  1.76956512]
 [26.1583545   1.13869124  2.68397242]
 [26.20779211  1.23202865  0.83720092]]
       Step     Time          Energy          fmax
LBFGS:    0 18:49:27    -4133.047388         0.003757
LBFGS:    1 18:49:27    -4133.047389         0.007336
LBFGS:    2 18:49:27    -4133.047388         0.000003
main.py 94: QC_coords1 [[ 6.44549715  3.64874579  3.31889449]
 [49.4788378   2.24000596  3.32702665]
 [ 7.72349269  4.84091458  3.29379405]
 [ 5.16750045  2.45657816  3.34399497]
 [49.43212303  2.15180921  5.07207914]
 [49.52555226  2.3282021   1.58197411]]


******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-12
SCF conv_tol_grad = None
SCF max_cycles = 100
direct_scf =

In [4]:
from MQGO import optimizer
Opt = optimizer.Optimizer(xyz_path="water0.xyz",
									work_path="./",
									ml_engine='deepmd',
									outter_cycle=1,
									algorithm='bfgs',
									max_opt_cycle=100)
atom = Opt.ase_read_xyz()
print(atom.get_positions())
atom.calc = DP(model='graph.pb')
print(atom.get_potential_energy())
force0_ml = atom.get_forces()
print(atom.get_forces())

[[ 3.63802  1.5711   1.76   ]
 [26.36198  1.5711   1.76   ]
 [ 3.95828  2.55412  1.74392]
 [ 2.63613  1.66728  1.76493]
 [26.09174  0.99418  2.56104]
 [26.0955   0.9908   0.96072]]
-4132.799365865741
[[ 1.74271480e+00 -4.02384015e+00  4.66008093e-02]
 [ 1.86615095e+00  4.02352386e+00 -1.68861396e-02]
 [ 5.33263437e-01  3.28007431e+00 -5.02850158e-02]
 [-2.27597824e+00  7.43765843e-01  3.68420650e-03]
 [-9.39618606e-01 -2.01139593e+00  2.02486619e+00]
 [-9.26532345e-01 -2.01212793e+00 -2.00798005e+00]]


In [5]:
import numpy as np
forces = np.loadtxt('raw/force.raw')
force0_qc = forces[0].reshape((6,3))
print(force0_qc)

[[ 1.73445043e+00 -4.04231476e+00  4.69214939e-02]
 [ 1.86311912e+00  4.01780156e+00 -1.71757040e-02]
 [ 5.53645372e-01  3.30830811e+00 -5.08197707e-02]
 [-2.28815565e+00  7.34015571e-01  3.89837807e-03]
 [-9.38164644e-01 -2.00861169e+00  2.04263408e+00]
 [-9.24894634e-01 -2.00919880e+00 -2.02545848e+00]]


In [6]:
delta = force0_qc-force0_ml
with open('debug_force.txt', 'a') as file:
    for i in range(0, len(force0_qc)):
        for j in range(0, 3):
            file.write('%24.18f'%delta[i][j])
        file.write('\n')
    file.write('\n')

In [7]:
print(max(delta.flatten()))

0.028233807265405364
